## Introduction
This file contains the data transformations applied for the visualization in the first page of the website. It is assumed that the data are downloaded and stored in the 'data' directory of the root project folder. Pandas package is required for running this notebook.

Author: Mahmudul Hasan Bhuiyan

In [112]:
import pandas as pd
from datetime import datetime

In [113]:
#file paths and other necessary variables
input_index_data_path = '../data/compounded_index.csv'
output_index_data_path = '../data/Compounded_index_transformed.csv'
ci_index_path = '../data/OxCGRT_latest.csv'
wp_file_path ='../data/c2_workplace_closing.csv'
sh_file_path ='../data/c6_stay_at_home_requirements.csv'
tr_file_path ='../data/c8_internationaltravel.csv'

type_dict = {'Open':float, 'High':float, 'Low':float}

## Stock Index data file transformations

Read and clean the stock index file, create a new column named 'Avg' representing the mean of 'High' and 'Low' column. 'M' in the volume column of European index is converted to Million and for few missing values (-), a zero (0) is put.

In [121]:
def create_avg_column(filepath):
    data  = pd.read_csv(filepath,index_col=0, thousands=',', dtype=type_dict, parse_dates=['Date'])
    #fix the data representation issues
    data['Volume'] = data[data['Index'] == 'STOXX 50 EU'].Volume.str.replace("M", "")
    data['Volume'] = data[data['Index'] == 'STOXX 50 EU'].Volume.str.replace("-", "0")
    data['Volume'] = data['Volume'].astype(float)
    data['Volume'] = data['Volume']*1000000
    data['Volume'] = data['Volume'].astype(int)
    data['Close'] = data['Close'].astype(float)
    data['Avg'] = data[['High', 'Low']].mean(axis=1)
    data['Index'] = data['Index'].str.replace("Hang Sang", "Hang Seng")
    
    return data

#cread, clean and save the stock index file, create a new column named 'Avg'
df = create_avg_column(index_data_path)
df.to_csv(output_index_data_path,index=False)

## Transformations for Confinement Index
    
The dataset(https://github.com/OxCGRT/covid-policy-tracker) contains 
indicators(https://github.com/OxCGRT/covid-policy-tracker/blob/master/documentation/codebook.md) of 
    
    C - containment and closure policies
    E - economic policies
    H - health system policies
    M - miscellaneous policies
    
From which, we only consider the containment indicators. It contains data for all the countries in national and state level, but we take only the national level data of respective countries of different stock indices.

In [115]:
def get_early_date_ci_df():
    
    '''
    In the dataset, the confinement measures are available from 2020-01-01. As we have stock data
    from September 2019, to align both the datasets, we create and add new rows from September 19 
    where there is no restrictions.
    '''
    
    early_dates = pd.date_range(start="2019-09-01",end="2019-12-31").strftime("%Y-%m-%d").tolist()
    
    indices = ['Hang Seng', 'S&P 500', 'STOXX 50 EU']
    
    early_date_rows = []
    
    for date in early_dates:
        for index in indices:
            early_date_rows.append([date, index, 0.0])
    
    early_df = pd.DataFrame(early_date_rows)
    early_df.columns = ['Date', 'Index', 'CI']
    
    return early_df

def get_df(path):
    
    '''
    Read the data, apply some transformations and return the dataframe. 
    
    '''
    #read the csv file
    dateparse = lambda x: datetime.strptime(x, '%Y%m%d')
    data  = pd.read_csv(path, parse_dates=['Date'], date_parser=dateparse, low_memory=False)

    #filter only the national level data
    data_national = data[data.Jurisdiction=='NAT_TOTAL']

    #drop unnecessary columns
    df_confinement = data_national[ ['Date'] + list(data_national.loc[:,'CountryName':'CountryCode']) + list(data_national.loc[:,'C1_School closing':'C8_International travel controls']) ]
    
    #filter the countries
    countries = ['China', 'Hong Kong', 'South Korea', 'Singapore', 'Germany', 'Spain', 'France', 'Italy', 'Netherlands', 'United Kingdom', 'United States']
    df_confinement = df_confinement[df_confinement.CountryName.isin(countries)]
    
    return df_confinement


def get_index(row):
    
    '''
    Given a row of the dataframe, returns the corresponding stock index.
    '''
    
    index_countries = {'Hang Seng': ['China', 'Hong Kong', 'South Korea', 'Singapore'], 'STOXX 50 EU':['Germany', 'Spain', 'France', 'Italy', 'Netherlands', 'United Kingdom'], 'S&P 500':['United States']}
    country_name = row.CountryName
    for index in index_countries:
        countries = index_countries[index]
        if country_name in countries:
            return index
        

def get_confinement_index_df(filepath, is_aligned):
    
    '''
    Create a new column named as CI (Confinement Index) by averaging the values of 8 confinement indicators. 
    
    Params:
        filepath: path of the 'OxCGRT_latest' file
        is_aligned: bollean variable to indicate wheather to align the data with the stock market data 
            by creating dates from September 2019 as specified before. 

    '''
    
    df = get_df(filepath)
    
    df = df[['Date', 'CountryName', 'CountryCode', 'C1_School closing', 'C2_Workplace closing', 'C3_Cancel public events', 'C4_Restrictions on gatherings', 'C5_Close public transport', 'C6_Stay at home requirements', 'C7_Restrictions on internal movement', 'C8_International travel controls']]
    df['Index'] = df.apply(lambda row: get_index(row), axis=1)

    df['CI'] = df.loc[:, 'C1_School closing':'C8_International travel controls'].mean(axis=1)
    df_ci = df[['Date', 'Index', 'CI']]
    
    #we drop the null values 
    df_ci = df_ci.dropna()

    df_ci = df_ci.groupby(['Date', 'Index']).mean()
    df_ci.reset_index(inplace=True)
    
    #if data has to be alligned with the stock market data
    if is_aligned:
        early_df = get_early_date_ci_df()
        df_ci = pd.concat([early_df, df_ci], ignore_index=True)
    
    #convert the string date to pandas date format
    df_ci['Date'] = pd.to_datetime(df_ci['Date'], format='%Y-%m-%d')
    
    # We take data until the end of 2020
    df_ci = df_ci[df_ci['Date'] < '2021-01-01']
    df_ci = df_ci.round({'CI': 4})
    return df_ci

df = get_confinement_index_df(filepath=ci_index_path, is_aligned=True)
df.to_csv('../data/Confinement_index_transformed.csv')
print('Done')

Done


## Transformations for Confinement Measures

Although the confiment index is calculated over all the confinement indicators, to visualize the actual measures 
we only consider the most important confinement indicators representing Workplace closing(C2), Stay at home(C6) and International travel measures (C8) (https://github.com/OxCGRT/covid-policy-tracker/tree/master/data/timeseries). 

As there are some enforced confinement measures since the early days of Covid, instead of taking all the dates we only take the dates on which there is a change in the confinement policy. 

To visualize the confinement measures, we only consider the economically most important countries of EU (Germany, Spain, France, Italy, Netherlands, United Kingdom) among all the countries comprising the STOXX 50 EU index.

In [116]:
def get_confinement_df(path):
    
    '''
    Read the file confinement file and create a dataframe with the countries under consideration.
    '''
    
    df = pd.read_csv(path)
    #filter the countries
    countries = ['China', 'Hong Kong', 'South Korea', 'Singapore', 'Germany', 'Spain', 'France', 'Italy', 'Netherlands', 'United Kingdom', 'United States']
    df = df[df.country_name.isin(countries)]
    df = df.dropna(axis=1)

    df = df.T
    df = df.iloc[3:]
    df = df.reset_index()
    df.columns = ['Date', 'China', 'Germany', 'Spain', 'France', 'United Kingdom',
       'Hong Kong', 'Italy', 'South Korea', 'Netherlands', 'Singapore',
       'United States']
    df['Date'] =  pd.to_datetime(df['Date'], format='%d%b%Y')
    
    return df


def get_combined_confinement_df(wp_file=wp_file_path, sh_file=sh_file_path, tr_file=tr_file_path):
    
    '''
    Combines the three confinement indicator values in a dataframe and returns it
    '''
    
    workplace_df = get_confinement_df(wp_file)
    stayhome_df = get_confinement_df(sh_file)
    travel_df = get_confinement_df(tr_file)
    
    combined_df = pd.DataFrame()
    combined_df['Date'] = workplace_df.Date.copy()
    
    columns = workplace_df.columns.tolist()
    columns.remove('Date')

    for column in columns:
        combined_df[column] = workplace_df[column].astype(int).apply(str)+stayhome_df[column].astype(int).apply(str)+travel_df[column].astype(int).apply(str)
    return combined_df


def convert_text_instruction(instruction_code):
    
    '''
    Convert the confinement indicator values to text measures accordign to the encoding of data source
    '''

    instruction_map = {
        'wp':{'1':'Recommended workplace closing', '2':'Require workplace closing for some sectors', '3': 'Require closing for all but essential workplaces'},
        'sh':{'1':'Recommended not leaving house', '2':'Require not leaving house with exceptions for essential trips', '3':'Require not leaving house with minimal exceptions'},
        'tr':{'1':'Covid screening on arrivals', '2':'Quarantine on arrivals from some or all regions', '3':'Ban on arrivals from some regions', '4':'Ban on all regions or total border closure'}
    }
    
    if instruction_code == '000':
        return 'No restriction'
    else:
        wpc = instruction_code[0]
        shc = instruction_code[1]
        trc = instruction_code[2]
        
        instruction_text = ''
        
        #
        if wpc != '0':
            instruction_text += instruction_map['wp'][wpc]
        
        if shc != '0':
            if instruction_text == '':
                instruction_text += instruction_map['sh'][shc]
            else:
                instruction_text += '; '+instruction_map['sh'][shc]
        
        if trc != '0':
            if instruction_text == '':
                instruction_text += instruction_map['tr'][trc]
            else:
                instruction_text += '; '+instruction_map['tr'][trc]
        
    return instruction_text

    
def get_text_confinement_df():
    
    '''
    Applies the function for converting the instruction code to text measures
    '''
    
    df = get_combined_confinement_df()
    columns = df.columns.tolist()
    columns.remove('Date')
    
    for column in columns:
        df[column] = df[column].apply(convert_text_instruction)

    return df


def remove_unchanged_rows(df):
    
    '''
    For visualization purpose, we only keep the dates on which there is a change in the measures.
    This function removes the date between two measure change dates.
    '''
    
    prev_row = None
    keep_index = []
    columns = df.columns.tolist()
    columns.remove('Date')
    
    for index, row in df.iterrows():
        
        if index == 0:
            prev_row = row
            keep_index.append(index)
            continue
            
        for col in columns:
            if prev_row.loc[col] != row.loc[col]:
                keep_index.append(index)
                prev_row = row
            break
    
    df = df.loc[keep_index]
    
    return df


def combine_index_measure(df, index_name):
    
    '''
    Combines the measures of countries corresponding to a particular stock index to a single 
    measure string. 
    '''
    
    row_list = []
    row_list.append(["2019-09-01", index_name, "No restriction"])
    
    columns = df.columns.tolist()
    columns.remove('Date')
    
    for index, row in df.iterrows():
        measure = "\n"
        for col in columns:
            measure += col + ": " + row.loc[col] + '\n'
        row_list.append([row.loc['Date'].strftime("%Y-%m-%d"), index_name, measure])
    
    result_df = pd.DataFrame(row_list)
    result_df.columns = ['Date', 'Index', 'Measure']
    result_df['Date'] = pd.to_datetime(result_df['Date'], format='%Y-%m-%d')
    return result_df


def get_indexed_measure():
    
    '''
    Creates the textual confinement measures for countries of different stock indices.
    It only cpatures the date on which there is a chaneg in the confinement rules.    
    '''
    
    df = get_text_confinement_df()
    
    asian_index_df = df[['Date', 'China', 'Hong Kong', 'South Korea', 'Singapore']]
    eu_index_df = df[['Date', 'Germany', 'France', 'Spain', 'Italy', 'Netherlands', 'United Kingdom']]    
    usa_index_df = df[['Date', 'United States']]
    
    asian_index_df = remove_unchanged_rows(asian_index_df)
    eu_index_df = remove_unchanged_rows(eu_index_df)   
    usa_index_df = remove_unchanged_rows(usa_index_df)
    
    asian_index_df = combine_index_measure(asian_index_df, "Hang Seng")
    eu_index_df = combine_index_measure(eu_index_df, "STOXX 50 EU")   
    usa_index_df = combine_index_measure(usa_index_df, "S&P 500")
    
    df = pd.concat([asian_index_df, eu_index_df, usa_index_df])
    df.reset_index(inplace=True, drop=True)
    return df
    

df = get_indexed_measure()
df.to_csv('../data/Confinement_measures_transformed.csv')